# NLP with NLTK (.. and a little sklearn) 

Natural Language Processing with the Natural Language Toolkit

[nltk](http://www.nltk.org/) is a Python package for NLP.

In [1]:
from __future__ import print_function

In [2]:
# pip install nltk
import nltk
import pandas as pd

Much of NLTK depends on additional data which you'll have to download. Use `nltk.download()` to get at least the following:

 * averaged_perceptron_tagger (in models)
 * maxent_treebank_pos_tagger (in models)
 * punkt (in models)
 * maxent_ne_chunk (in models)
 * words (in corpora)

You can install these and continue without restarting your kernel.

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

### Sentence tokenization

In [ ]:
from nltk.tokenize import sent_tokenize

text = """Hello. How are you, dear Mr. Sir? Are you well?
          Here: drink this! It will make you feel better.
          I mean, it won't make you feel worse!"""

sentences = sent_tokenize(text)
print(sentences)


### Word tokenization

In [ ]:
# TreebankWordTokenizer assumes that our input has already been segmented into sentences..


from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(sentences[5])

In [ ]:
from nltk.tokenize import word_tokenize
words = word_tokenize(sentences[5])
words

In [ ]:
from nltk.tokenize import wordpunct_tokenize
wordpunct_tokenize(sentences[5])

Demo of different tokenizers: http://text-processing.com/demo/tokenize/

### Part of speech tagging

In [ ]:
from nltk.tag import pos_tag
words=pos_tag(word_tokenize("Who's going to that thing today?"))
words

##### Some of POS tags: 
WP: wh-pronoun ("who", "what")  
VBZ: verb, 3rd person sing. present ("takes")  
VBG: verb, gerund/present participle ("taking")  
TO: to ("to go", "to him")   
DT: determiner ("the", "this")  
NN: noun, singular or mass ("door")  
.: Punctuation (".", "?")  

All tags: http://www.monlp.com/2011/11/08/part-of-speech-tags/

### Chunking
Extracting phrases

In [ ]:
## the 'named entity' chunker!  ne_chunk utilizes 

from nltk.chunk import ne_chunk
words = word_tokenize("""I'm Julia and I'm here to say
                         I love NLTK in a major way.""")
tags = pos_tag(words)
tree = ne_chunk(tags)
print(tags)
print(tree)

In [ ]:
tree.draw()

In [ ]:
words2 = word_tokenize("First National Bank announced earnings!")
tags2=pos_tag(words2)
tree2=ne_chunk(tags2)

In [ ]:
tree2.draw()

### Included text corpora

Also install these!

 * movie_reviews: Imdb reviews characterized as pos & neg  
 * treebank: tagged and parsed Wall Street Journal text  
 * brown: tagged & categorized English text (news, fiction, etc)  

(There are over 60 others.)

In [ ]:
nltk.download()

In [ ]:
# A chunked corpora reader..

from nltk.corpus import treebank_chunk
treebank_chunk.tagged_sents()[0]

In [ ]:
treebank_chunk.chunked_sents()[0].draw()

In [ ]:
tree=treebank_chunk.chunked_sents()[0]
' '.join([w for w, t in tree.leaves()])

# TextBlob

In [ ]:
# pip install textblob
from textblob import TextBlob

GATSBY_TEXT = """In my younger and more vulnerable years my father
                 gave me some advice that I've been turning over
                 in my mind ever since. "Whenever you feel like
                 criticizing any one," he told me, "blah blah blah."""

gatsby = TextBlob(GATSBY_TEXT)

In [ ]:
gatsby.tags

In [ ]:
gatsby.noun_phrases

###  How do you really feel?    TextBlob:  Sentiment Analysis

In [ ]:
TextBlob("Oh my god I love this bootcamp, it's so awesome.").sentiment

In [ ]:
TextBlob("it's so awesome").sentiment

In [ ]:
TextBlob("Oh my god.").sentiment

In [ ]:
TextBlob("I love this bootcamp.").sentiment

In [ ]:
TextBlob("Oh my god I love this bootcamp.").sentiment

In [ ]:
TextBlob("it's so awesome.").sentiment

In [ ]:
print(TextBlob("I hate cupcakes.").sentiment)

In [ ]:
gatsby.sentences

In [ ]:
gatsby.words

In [ ]:
gatsby.sentences[0].words

#### Stemming

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()
for word in TextBlob("Are you running in two marathons?").words:
    print(stemmer.stem(word))

To see different nltk stemmers in effect:
http://text-processing.com/demo/stem/

In [ ]:
for word, count in gatsby.word_counts.items():
    print("%15s %i" % (word, count))

In [ ]:
def get_count(item):
    return item[1]

for word, count in sorted(gatsby.word_counts.items(), key=get_count, reverse=True):
    print("%15s %i" % (word, count))

### Movie Reviews 
(without stopwords!)

In [ ]:
nltk.download()

In [ ]:
import nltk
from textblob import TextBlob
from nltk.corpus import movie_reviews

fileids = movie_reviews.fileids()[:100]
doc_words = [movie_reviews.words(fileid) for fileid in fileids]
documents = [' '.join(words) for words in doc_words]
print(documents[0:1])

##### Top bigrams in reviews

In [ ]:
from nltk.util import ngrams

from collections import Counter
from operator import itemgetter

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"']
stop = set(stop)

counter = Counter()

n = 2
for doc in documents:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stop]
    bigrams = ngrams(words, n)
    counter += Counter(bigrams)

for phrase, count in counter.most_common(30):
    print('%20s %i' % (" ".join(phrase), count))

### Using Sklearn algorithms with text data

CountVectorizer:  Convert a collection of text documents to a matrix of token counts
This implementation produces a sparse representation.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text = ['That is should come to this!', 'This above all: to thine own self be true.', 'Something is rotten in the state of Denmark.']

# CountVectorizer is a class; so `vectorizer` below represents an instance of that object.
vectorizer = CountVectorizer(ngram_range=(1,2))

# call `fit` to build the vocabulary
vectorizer.fit(text)

# then, use `get_feature_names` to return the tokens
print(vectorizer.get_feature_names())

# finally, call `transform` to convert text to a bag of words
x = vectorizer.transform(text)

In [ ]:
print('Sparse Matrix')
# A compressed version; the "sparse" matrix.
print(type(x))
print(x)

print ('Matrix')
x_back = x.toarray()
print(type(x_back))
print(x_back)

In [ ]:
pd.DataFrame(x_back, columns=vectorizer.get_feature_names())

In [ ]:
#### TF: frequency in this document
#### IDF: inverse frequency in the corpus

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
doc_vectors = vectorizer.fit_transform(documents)

classes = np.array(['pos']*50 + ['neg']*50)


model = MultinomialNB().fit(doc_vectors, classes)

In [ ]:
print(GATSBY_TEXT)

In [ ]:
gatsby_vector = vectorizer.transform([GATSBY_TEXT])
model.predict(gatsby_vector)